In [14]:
import cv2
import os
import os.path as osp
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd

In [3]:
def get_frames(path, fps=None):
    cap = cv2.VideoCapture(path)

    time_increment = (
        1/fps 
        if fps is not None 
        else None
    )

    res = []
    success = 1
    sec = 0
    while success:
        if time_increment is not None:
            sec += time_increment
            cap.set(cv2.CAP_PROP_POS_MSEC, 1000*sec)
        success, image = cap.read()
        if success:
            res.append(image)

    return res


def save_frames(
    frames, 
    dir, 
    prefix="frame"
):
    for i, frame in enumerate(frames):
        cv2.imwrite(str(osp.join(dir, f"{prefix}_{i}.jpg")), frame)


def to_gray(frames):
    res = [cv2.cvtColor(f, cv2.COLOR_BGR2GRAY) for f in frames]
    return res


def to_scale(frames, scale=0.5):
    res = []
    for f in frames:
        width = int(f.shape[1] * scale)
        height = int(f.shape[0] * scale)
        dim = (width, height)
        resized = cv2.resize(f, dim, interpolation=cv2.INTER_AREA)
        res.append(resized)
        
    return res


def read_pickle(dir):
    with open(dir, 'rb') as handle:
        b = pickle.load(handle)
    return b


def write_pickle(dir, data):
    with open(dir, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

def to_pickle_name(file):
    return file.split(".")[0] + ".pkl"


def flatten(frames):
    return [f.flatten() for f in frames]

In [18]:
def to_fps_gray_scale(
    base_dir,
    save_dir,
    files,
    fps,
    scale,
    overwrite=False,
):
    for file in files:
        save_file = osp.join(save_dir, to_pickle_name(file))
        if osp.exists(save_file) and not overwrite:
            continue
        
        path = osp.join(base_dir, file)
        res = get_frames(path, fps)
        gray = to_gray(res)
        resized_gray = to_scale(gray, scale)
        
        write_pickle(save_file, resized_gray)
        

def get_files(dir, format="mp4"):
    res = []
    for filename in os.listdir(dir):
        if filename.split(".")[-1] == format:
            res.append(filename)
    return res


def to_actions(f):
    words = f.split("_")
    res = []
    for w in words:
        if w.startswith("y") and w[1:].isnumeric():
            break
        res.append(w)
    
    return res


def to_df(
    pickle_dir,
    pickle_files
):
    res = []
    for f in pickle_files:
        frames = read_pickle(osp.join(pickle_dir, f))
        frames = flatten(frames)
        actions = to_actions(f)
        count = len(actions)
        res.append((frames, actions, count))

    df = pd.DataFrame(data=dict(zip(["embedding", "label", "count"], np.transpose(res))))
    df.to_pickle(save_path)
    
    return df

In [19]:
base_dir = "C:/Users/aphri/Documents/t0002/pycharm/data/action_dataset0003"
save_dir = "C:/Users/aphri/Documents/t0002/pycharm/data/ar_fps2_gray_scale3/pickle"

In [ ]:
files = get_files(base_dir)
    
to_fps_gray_scale(
    base_dir=base_dir,
    save_dir=save_dir,
    files=files,
    fps=2,
    scale=0.3,
)

In [20]:
save_path = "C:/Users/aphri/Documents/t0002/pycharm/data/ar_fps2_gray_scale3/df.pkl"
pickle_files = get_files(save_dir, "pkl")

df = to_df(
    save_dir,
    pickle_files
)

c:\users\aphri\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [21]:
df.head()

,embedding,label,count
0,"[[87, 86, 86, 87, 86, 86, 86, 86, 86, 86, 85, ...","[backhandspring, backhandspring, backhandspring]",3
1,"[[239, 239, 238, 239, 238, 238, 238, 238, 237,...","[backhandspring, backhandspring, backhandspring]",3
2,"[[87, 86, 86, 87, 86, 86, 86, 86, 86, 86, 85, ...","[backhandspring, backhandspring, backhandspring]",3
3,"[[229, 229, 229, 229, 228, 228, 228, 228, 228,...","[backhandspring, backhandspring, backhandspring]",3
4,"[[87, 86, 86, 87, 86, 86, 86, 86, 86, 86, 85, ...","[backhandspring, backhandspring, backhandspring]",3


In [22]:
df.to_pickle(save_path)

In [23]:
df2 = pd.read_pickle(save_path)

In [24]:
df2.head()

,embedding,label,count
0,"[[87, 86, 86, 87, 86, 86, 86, 86, 86, 86, 85, ...","[backhandspring, backhandspring, backhandspring]",3
1,"[[239, 239, 238, 239, 238, 238, 238, 238, 237,...","[backhandspring, backhandspring, backhandspring]",3
2,"[[87, 86, 86, 87, 86, 86, 86, 86, 86, 86, 85, ...","[backhandspring, backhandspring, backhandspring]",3
3,"[[229, 229, 229, 229, 228, 228, 228, 228, 228,...","[backhandspring, backhandspring, backhandspring]",3
4,"[[87, 86, 86, 87, 86, 86, 86, 86, 86, 86, 85, ...","[backhandspring, backhandspring, backhandspring]",3
